# Ogum Software - Aplicação Principal

In [ ]:
import ipywidgets as widgets

from IPython.display import display, clear_output

from pathlib import Path



try:

    from modulo2_importacao import Modulo2Importacao

    from modulo3_filtrorecorte import Modulo3Recorte

    from modulo_logtheta import ModuloLogTheta

    from modulo6_0_arrhenius import Modulo6_0_Arrhenius

    from modulo5_1_alinhamento import Modulo5_1Alinhamento
    
    from final_report import FinalReportModule
    from ogum.mesh_generator_ui import MeshGeneratorUI

except Exception:

    # Optional modules may not be available when building docs/tests

    Modulo2Importacao = Modulo3Recorte = ModuloLogTheta = None

    Modulo6_0_Arrhenius = Modulo5_1Alinhamento = None

    FinalReportModule = None
    MeshGeneratorUI = None





class MainApp:

    def __init__(self):

        self.step = 0

        self.modules = {}

        self.dfs_from_mod2 = []
        self.mesh_path = None

        self._content = widgets.Output()

        self.progress = widgets.IntProgress(layout=dict(display="none", width="100%"))

        self.status_label = widgets.Label(layout=dict(display="none"))

        self.log_output = widgets.Output()

        self.log_panel = widgets.Accordion(children=[self.log_output])

        self.log_panel.set_title(0, "Ver/Ocultar Logs da Sessão")

        self.btn_next = widgets.Button(description="Avançar", button_style="success")

        self.btn_back = widgets.Button(description="Voltar")

        self.btn_next.on_click(self._on_next_step)

        self.btn_back.on_click(self._on_back_step)

        self.header = widgets.HBox([widgets.HTML("<h2>&#128296; Ogum Sintering</h2>")])

        self.footer = widgets.HBox([self.btn_back, self.btn_next])

        self.ui = widgets.VBox(

            [

                self.header,

                self._content,

                self.progress,

                self.status_label,

                self.log_panel,

                self.footer,

            ]

        )

        self._inject_style()

        self._show_welcome()



    def _show_welcome(self):

        with self._content:

            clear_output()

            display(widgets.HTML("<h2>Bem-vindo ao Ogum Software</h2>"))



    def _inject_style(self):

        try:

            css = Path("ogum/style.css").read_text()

            display(widgets.HTML(f"<style>{css}</style>"))

        except Exception as exc:

            print(f"Erro ao carregar estilo: {exc}")



    def _set_busy(self, busy: bool, message: str = ""):

        self.btn_next.disabled = busy

        self.btn_back.disabled = busy

        self.status_label.value = message

        self.progress.layout.display = "block" if busy else "none"

        self.status_label.layout.display = "block" if busy else "none"



    # Navigation callbacks

    def _on_next_step(self, _=None):

        self.step += 1

        self._render_step()



    def _on_back_step(self, _=None):

        if self.step > 0:

            self.step -= 1

        self._render_step()



    def _render_step(self):

        with self._content:

            clear_output()

            if self.step == 0:

                self._show_welcome()

            elif self.step == 1:

                if "mod2" not in self.modules and Modulo2Importacao:

                    self.modules["mod2"] = Modulo2Importacao()

                mod = self.modules.get("mod2")

                if mod:

                    display(mod.main_ui)

                    self.dfs_from_mod2 = getattr(mod, "dfs_ensaios", [])

            elif self.step == 2:

                if "mod3" not in self.modules and Modulo3Recorte:

                    self.modules["mod3"] = Modulo3Recorte(self.dfs_from_mod2)

                mod = self.modules.get("mod3")

                if mod:

                    display(mod.main_ui)

            elif self.step == 3:
                self._mesh_choice = widgets.ToggleButtons(
                    options=["Usar malha padrão", "Criar malha interativa"], description="Malha:"
                )
                self._mesh_choice.observe(self._on_mesh_selected, names="value")
                display(self._mesh_choice)
                self._mesh_box = widgets.Output()
                display(self._mesh_box)
            elif self.step == 4:
                self._method_choice = widgets.ToggleButtons(
                    options=["Curva Mestra", "Arrhenius"], description="Método:"
                )
                self._method_choice.observe(self._on_method_selected, names="value")
                display(self._method_choice)
                self._analysis_box = widgets.Output()
                display(self._analysis_box)
            
            elif self.step == 5:

                if "report" not in self.modules and FinalReportModule:

                    records = getattr(self.modules.get("mod5"), "records", [])

                    self.modules["report"] = FinalReportModule(

                        records,

                        on_busy=self._set_busy,

                        log_output=self.log_output,

                    )

                mod = self.modules.get("report")

                if mod:

                    display(mod.main_ui)

    def _on_mesh_selected(self, change):
        with self._mesh_box:
            clear_output()
            if change.new == "Criar malha interativa" and MeshGeneratorUI:
                if "meshui" not in self.modules:
                    self.modules["meshui"] = MeshGeneratorUI()
                display(self.modules["meshui"].ui)
                self.mesh_path = self.modules["meshui"].mesh_path
            else:
                self.mesh_path = None



    def _on_method_selected(self, change):

        with self._analysis_box:

            clear_output()

            if change.new == "Curva Mestra" and ModuloLogTheta:

                if "mod4" not in self.modules:

                    self.modules["mod4"] = ModuloLogTheta(self.dfs_from_mod2)

                display(self.modules["mod4"].main_ui)

                # Placeholder for subsequent modules until FinalReport

                if "mod5" not in self.modules and Modulo5_1Alinhamento:

                    self.modules["mod5"] = Modulo5_1Alinhamento(self.dfs_from_mod2)

            elif change.new == "Arrhenius" and Modulo6_0_Arrhenius:

                if "mod4" not in self.modules:

                    self.modules["mod4"] = Modulo6_0_Arrhenius(self.dfs_from_mod2)

                display(self.modules["mod4"].main_ui)





app = MainApp()

app.ui
